In [6]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # 현재 디렉터리의 상위 경로 추가
print(sys.path)

['c:\\Users\\user\\miniconda3\\envs\\deeplearning\\python311.zip', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\DLLs', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\Lib', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning', '', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\Lib\\site-packages', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\Lib\\site-packages\\win32', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\user\\miniconda3\\envs\\deeplearning\\Lib\\site-packages\\Pythonwin', 'c:\\wanted\\project_diary\\Wanted_DLproject', 'c:\\wanted\\project_diary\\Wanted_DLproject']


### GEMINI

In [5]:
import google.generativeai as genai
from config.api_keys import gemini_key

genai.configure(api_key=gemini_key)
model_list = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
print('\n'.join(model_list[:5]))

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest


In [33]:
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 줄바꿈은 최대한 적게 사용하여 짧게 응답하세요. 
사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고, 일기를 쉽게 작성할 수 있도록 도와주세요.
사용자가 답변을 한 경우에는 답변에 대해 사용자의 감정을 공감하고 친절하게 안내하는 어조로 하고 다음 줄에 연관된 객관식 문제를 출력하세요.
객관식 질문 정답에 대한 보기 문항은 4가지로 작성하세요.
객관식 질문 문항의 경우 문제 1줄, 객관식 보기 1줄로 작성하세요.
"""

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction
)

response = model.generate_content("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 5개 작성해줘.")
print(response.text)

오늘 하루 어떤 일이 있었는지 궁금하네요!  먼저, 오늘 하루를 한 단어로 표현한다면? ① 행복 ② 슬픔 ③ 평범 ④ 흥분

오늘 하루 중 가장 기억에 남는 순간은 무엇이었나요?  ① 친구와의 만남 ② 맛있는 음식 ③ 특별한 경험 ④ 아무것도 없음

그 순간 당신의 기분은 어땠나요? ① 행복하고 즐거웠다 ② 편안하고 평온했다 ③ 불안하고 초조했다 ④ 짜릿하고 신났다

그 기분을 느끼게 된 이유는 무엇인가요? ① 긍정적 사건 ② 부정적 사건 ③ 특별한 감정 ④ 아무런 이유 없음

오늘 하루 당신에게 가장 중요했던 것은 무엇이었나요? ① 관계 ② 성취 ③ 휴식 ④ 자기성찰



In [34]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction
)
chat_history = []  # 이전 대화내용 기억

def ask_gemini(prompt, max_history_length=5):
    """대화 기록을 유지하며 Gemini 모델과 대화"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 합쳐서 프롬프트 생성
    full_prompt = "\n".join(chat_history) + f"\nUser: {prompt}"

    # 모델 응답 생성
    response = model.generate_content(full_prompt)

    # 새로운 질문과 답변을 대화 기록에 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"Gemini: {response.text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > max_history_length * 2:
        chat_history = chat_history[-(max_history_length * 2):]

    return response.text

# 대화 예시
ask_gemini("오늘 하루에 대한 일기를 쓰려고 해.")
ask_gemini("1번을 선택할래.")
for chat in chat_history:
    print(chat, '\n----------------------')

User: 오늘 하루에 대한 일기를 쓰려고 해. 
----------------------
Gemini: 오늘 하루는 어떠셨나요?  힘들었던 하루였는지, 아니면 행복했던 하루였는지 궁금하네요.  

오늘 하루 당신의 기분을 가장 잘 나타내는 단어는 무엇인가요? ① 행복 ② 슬픔 ③ 평온 ④ 흥분

 
----------------------
User: 1번을 선택할래. 
----------------------
Gemini: 행복하셨다니 정말 다행이네요! 오늘 행복했던 순간들을 떠올리며 기분 좋게 일기를 써보세요.  어떤 순간이 가장 행복했나요?

오늘 당신이 가장 행복했던 순간은 무엇이었나요? ① 친구와의 만남 ② 맛있는 음식 ③ 좋아하는 취미생활 ④ 특별한 선물
 
----------------------


### Llama

In [6]:
from huggingface_hub import HfApi

# Hugging Face API 인스턴스 생성
api = HfApi()

# 'llama'를 이름에 포함하는 모델 목록 가져오기
models = api.list_models(filter="llama")
cnt = 0

# 추가 인증 없이 접근 가능한 모델 필터링 및 한국어 지원 여부 확인
for model in models:
    try:
        # 모델 카드 정보 가져오기
        model_info = api.model_info(model.modelId)
        
        # 게이트 처리된 모델은 접근 제한이 있으므로 제외
        if model_info.gated:
            continue

        # 모델 카드의 언어 정보 확인 (None 방지)
        languages = model_info.cardData.get("language", []) if model_info.cardData else []

        # 한국어 지원 여부 확인
        supports_korean = "ko" in languages or "Korean" in languages
        if not supports_korean:
            continue
        
        # 다운로드 수 확인
        downloads = model_info.downloads if hasattr(model_info, "downloads") else "정보 없음"
        if downloads < 10000:
            continue

        print(f"모델 ID: {model.modelId}")
        print(f"다운로드 수: {downloads}")
        print("-" * 40)

    except Exception as e:
        print(f"모델 {model.modelId} 처리 중 오류 발생: {e}")
        continue
    
    cnt += 1
    if cnt > 1000:
        break

모델 ID: NCSOFT/Llama-VARCO-8B-Instruct
다운로드 수: 11748
----------------------------------------
모델 ID: Bllossom/llama-3.2-Korean-Bllossom-3B
다운로드 수: 41400
----------------------------------------
모델 ID: beomi/Llama-3-Open-Ko-8B
다운로드 수: 18213
----------------------------------------
모델 ID: MLP-KTLim/llama-3-Korean-Bllossom-8B
다운로드 수: 35577
----------------------------------------


Invalid model-index. Not loading eval results into CardData.


모델 ID: haoranxu/X-ALMA-13B-Pretrain
다운로드 수: 14474
----------------------------------------
모델 ID: QuantFactory/DarkIdol-Llama-3.1-8B-Instruct-1.2-Uncensored-GGUF
다운로드 수: 21514
----------------------------------------


KeyboardInterrupt: 

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 로드
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# pad_token 문제 해결
tokenizer.pad_token = tokenizer.eos_token

# 시스템 프롬프트 설정
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고,
일기를 쉽게 작성할 수 있도록 도와주세요. 사용자가 답변을 한 경우, 그에 대한 감정을 공감하고 새로운 질문을 제공하세요.
객관식 질문은 4개의 보기 문항을 포함하며, 새로운 질문을 연속적으로 이어가도록 하세요.
"""

# 대화 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 기록 저장 개수

def ask_llama(prompt):
    """LLaMA 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    messages = [{"role": "system", "content": system_instruction}]  # 시스템 메시지 추가
    for i in range(0, len(chat_history), 2):
        messages.append({"role": "user", "content": chat_history[i].replace("User: ", "")})
        messages.append({"role": "assistant", "content": chat_history[i + 1].replace("LLaMA: ", "")})
    
    # 사용자의 새 질문 추가
    messages.append({"role": "user", "content": prompt})

    # 토크나이저 적용
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 종료 토큰 설정
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 모델 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # 새로운 대화 기록 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"LLaMA: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_llama("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 1개 작성해줘."))

# 사용자가 선택했을 때의 응답 테스트
print(ask_llama("1번을 선택했어. 이에 대한 감정을 공감하면서, 다음 질문을 제시해줘."))  # 답변 반영

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")


c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--MLP-KTLim--llama-3-Korean-Bllossom-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.55s/it]
Some parameters a

오늘 하루는 어떤 날이었나요?
1. 행복한 날
2. 힘든 날
3. 평범한 날
4. 기대되는 날

(당신이 선택한 답에 따라 다음 질문을 제공합니다.)
행복한 날을 보냈군요! 오늘 행복을 느꼈던 순간을 떠올려보세요.
1. 친구와 함께 시간을 보냈을 때
2. 일과 관련된 좋은 소식이 들렸을 때
3. 자연을 감상하면서 느꼈을 때
4. 취미 활동을 하면서 느꼈을 때

(당신이 선택한 답에 따라 다음 질문을 제공합니다.)

=== 대화 내역 ===
User: 오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 1개 작성해줘. 
----------------------
LLaMA: 오늘 하루는 어떤 날이었나요?
1. 행복한 날
2. 힘든 날
3. 평범한 날
4. 기대되는 날

(당신이 선택한 답에 따라 다음 질문을 제공합니다.) 
----------------------
User: 1번을 선택했어. 이에 대한 감정을 공감하면서, 다음 질문을 제시해줘. 
----------------------
LLaMA: 행복한 날을 보냈군요! 오늘 행복을 느꼈던 순간을 떠올려보세요.
1. 친구와 함께 시간을 보냈을 때
2. 일과 관련된 좋은 소식이 들렸을 때
3. 자연을 감상하면서 느꼈을 때
4. 취미 활동을 하면서 느꼈을 때

(당신이 선택한 답에 따라 다음 질문을 제공합니다.) 
----------------------


In [3]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# pad_token 문제 해결
tokenizer.pad_token = tokenizer.eos_token

# 시스템 프롬프트 설정
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고,
일기를 쉽게 작성할 수 있도록 도와주세요. 사용자가 답변을 한 경우, 그에 대한 감정을 공감하고 새로운 질문을 제공하세요.
객관식 질문은 4개의 보기 문항을 포함하며, 새로운 질문을 연속적으로 이어가도록 하세요.
"""

# 대화 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 기록 저장 개수

def ask_llama(prompt):
    """LLaMA 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    messages = [{"role": "system", "content": system_instruction}]  # 시스템 메시지 추가
    for i in range(0, len(chat_history), 2):
        messages.append({"role": "user", "content": chat_history[i].replace("User: ", "")})
        messages.append({"role": "assistant", "content": chat_history[i + 1].replace("LLaMA: ", "")})
    
    # 사용자의 새 질문 추가
    messages.append({"role": "user", "content": prompt})

    # 토크나이저 적용
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 종료 토큰 설정
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 모델 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # 새로운 대화 기록 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"LLaMA: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_llama("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 1개 작성해줘."))

# 사용자가 선택했을 때의 응답 테스트
print(ask_llama("1번을 선택했어. 이에 대한 감정을 공감하면서, 다음 질문을 제시해줘."))  # 답변 반영

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")

c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\accelerate\utils\modeling.py:1536: UserWarning: Current model requires 32.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

# KULLM

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

MODEL_DIR = "nlpai-lab/KULLM3"
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

s = "고려대학교에 대해서 알고 있니?"
conversation = [{'role': 'user', 'content': s}]
inputs = tokenizer.apply_chat_template(
    conversation,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt').to("cuda")
_ = model.generate(inputs, streamer=streamer, max_new_tokens=1024, use_cache=True)

c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  7.48it/s]
c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--nlpai-lab--KULLM3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activa

네, 고려대학교에 대해 알고 있습니다. 고려대학교는 대한민국 서울에 위치한 사립 대학교로, 1905년에 설립되었습니다. 이 대학교는 한국에서 가장 오래된 대학 중 하나로, 다양한 학부 및 대학원 프로그램을 제공합니다. 고려대학교는 특히 법학, 경제학, 정치학, 사회학, 문학, 과학 분야에서 높은 명성을 가지고 있습니다. 또한, 스포츠 분야에서도 활발한 활동을 보이며, 대한민국 대학 스포츠에서 중요한 역할을 하고 있습니다. 고려대학교는 국제적인 교류와 협력에도 적극적이며, 전 세계 다양한 대학과의 협력을 통해 글로벌 경쟁력을 강화하고 있습니다.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# 모델 로드
MODEL_DIR = "nlpai-lab/KULLM3"
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# pad_token 설정 (경고 방지)
tokenizer.pad_token = tokenizer.eos_token

# 대화 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 저장 개수

# 시스템 프롬프트 설정
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고,
일기를 쉽게 작성할 수 있도록 도와주세요. 사용자가 답변을 한 경우, 그에 대한 감정을 공감하고 새로운 질문을 제공하세요.
객관식 질문은 4개의 보기 문항을 포함하며, 새로운 질문을 연속적으로 이어가도록 하세요.
"""

def ask_kullm3(prompt):
    """KULLM3 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    messages = [{"role": "system", "content": system_instruction}]
    
    for i in range(0, len(chat_history), 2):
        messages.append({"role": "user", "content": chat_history[i].replace("User: ", "")})
        messages.append({"role": "assistant", "content": chat_history[i + 1].replace("AI: ", "")})

    # 새로운 사용자 입력 추가
    messages.append({"role": "user", "content": prompt})

    # 토크나이저 적용
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # 모델 응답 생성
    model.eval()
    outputs = model.generate(inputs, streamer=streamer, max_new_tokens=512, use_cache=True)
    response_text = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)

    # 대화 기록 업데이트
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"AI: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_kullm3("오늘 하루에 대한 일기를 쓰려고 해."))
print(ask_kullm3("1번을 선택할래."))  # 답변 반영

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")


c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  6.26it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


물론이죠! 오늘 하루에 대한 일기를 쓰는 데 도움을 드리겠습니다. 아래는 오늘 하루를 되돌아보며 답변할 수 있는 객관식 질문입니다.

### 질문 1
오늘 하루를 시작하기 전에 가장 기대했던 일은 무엇이었나요?

A) 친구와의 만남
B) 새로운 프로젝트 시작
C) 취미 활동
D) 휴식 시간

### 질문 2
오늘 하루 중 가장 기쁜 순간은 언제였나요?

A) 아침 커피 시간
B) 점심 식사 후의 산책
C) 저녁 식사 시간
D) 취미 활동 중

### 질문 3
오늘 하루 중 어려움을 겪었던 순간은 언제였나요?

A) 아침 출근 시간
B) 점심 시간
C) 오후 업무 시간
D) 저녁 시간

### 질문 4
오늘 하루를 통해 배운 가장 중요한 교훈은 무엇인가요?

A) 시간 관리의 중요성
B) 건강한 식습관의 중요성
C) 사람들과의 관계 유지의 중요성
D) 자기 자신에 대한 이해의 중요성

### 질문 5
오늘 하루를 마무리하며 내일을 위해
물론이죠! 오늘 하루에 대한 일기를 쓰는 데 도움을 드리겠습니다. 아래는 오늘 하루를 되돌아보며 답변할 수 있는 객관식 질문입니다.

### 질문 1
오늘 하루를 시작하기 전에 가장 기대했던 일은 무엇이었나요?

A) 친구와의 만남
B) 새로운 프로젝트 시작
C) 취미 활동
D) 휴식 시간

### 질문 2
오늘 하루 중 가장 기쁜 순간은 언제였나요?

A) 아침 커피 시간
B) 점심 식사 후의 산책
C) 저녁 식사 시간
D) 취미 활동 중

### 질문 3
오늘 하루 중 어려움을 겪었던 순간은 언제였나요?

A) 아침 출근 시간
B) 점심 시간
C) 오후 업무 시간
D) 저녁 시간

### 질문 4
오늘 하루를 통해 배운 가장 중요한 교훈은 무엇인가요?

A) 시간 관리의 중요성
B) 건강한 식습관의 중요성
C) 사람들과의 관계 유지의 중요성
D) 자기 자신에 대한 이해의 중요성

### 질문 5
오늘 하루를 마무리하며 내일을 위해
1번을 선택하셨군요. 그럼 1번에 대한 답변을 드리겠습니다. 1번은 "시간 관리의 중요성"입니다.

### DeepSeek

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 설정
model_name = "deepseek-ai/deepseek-r1-distill-qwen-7b"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드 (float16 또는 bfloat16 사용 가능)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,  # 커스텀 코드 실행 허용
    torch_dtype=torch.bfloat16,  # FP8 대신 bfloat16 사용 (VRAM 절약 가능)
    device_map="auto"  # 자동으로 GPU/CPU 설정
)

# pad_token 문제 해결
tokenizer.pad_token = tokenizer.eos_token

# 시스템 프롬프트 설정
system_instruction = """당신은 창의적이고 감성적인 작가입니다. 
사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고, 
일기를 쉽게 작성할 수 있도록 도와주세요. 
사용자가 답변을 한 경우, 그에 대한 감정을 공감하고 새로운 질문을 제공하세요. 
객관식 질문은 4개의 보기 문항을 포함하며, 새로운 질문을 연속적으로 이어가도록 하세요.
"""

# 대화 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 기록 저장 개수

def ask_deepseek(prompt):
    """DeepSeek-R1 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    messages = [{"role": "system", "content": system_instruction}]  # 시스템 메시지 추가
    
    for i in range(0, len(chat_history), 2):
        messages.append({"role": "user", "content": chat_history[i].replace("User: ", "")})
        messages.append({"role": "assistant", "content": chat_history[i + 1].replace("DeepSeek: ", "")})
    
    # 사용자의 새 질문 추가
    messages.append({"role": "user", "content": prompt})

    # 토큰 변환
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 종료 토큰 설정
    eos_token = tokenizer.eos_token_id

    # 모델 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=eos_token,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 응답 디코딩
    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # 새로운 대화 기록 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"DeepSeek: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_deepseek("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 2개 작성해줘."))

# 사용자가 선택했을 때의 응답 테스트
print(ask_deepseek("1번을 선택했어. 이에 대한 감정을 공감하면서, 다음 질문을 제시해줘."))  # 답변 반영

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")


c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--deepseek-ai--deepseek-r1-distill-qwen-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.15s/it]
Some parameter

Okay, the user wants to write a daily journal and needs two objective questions to help. I should come up with questions that encourage reflection and creativity. Let me think about topics that are meaningful and can inspire the user.

First, the purpose of the world is a big question. It makes the user think about their own impact and the bigger picture. It's deep but important.

Next, reflecting on the past three days can help them assess their mood and identify patterns. It's a practical approach to personal growth.

I should present these as options, so the user can choose which ones to use. Then, I'll invite them to share their answers and ask follow-up questions to keep the conversation going.
</think>

물론이에요! 일기를 쓰는 데 도움이 될 객관식 질문 2개를 VOD (Video Object Description)로 제안해드리겠습니다:

1. 오늘의 일기를 쓰기 위해 가장 중요한 점은 무엇인가요?  
2. 오늘의 일기를 쓰기 위해 가장 힘든 점은 무엇인가요?  

이 Questions를 VOD로 보여드리겠습니다.  
물론이에요! 일기를 쓰기 위해 가장 중요한 점은 무엇인가요?  
1. 오늘의 일기를 쓰기 위해 가장 중요한 점은 무엇인가요?  
2. 오늘의 일기를 쓰기 위해 가장 힘든 점은 무엇인가

### Llama VARCO

In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# pad_token 문제 해결
tokenizer.pad_token = tokenizer.eos_token

# 시스템 프롬프트 설정
system_instruction = """당신은 창의적이고 감성적인 작가입니다. 
사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고, 
일기를 쉽게 작성할 수 있도록 도와주세요. 
사용자가 답변을 한 경우, 그에 대한 감정을 공감하고 새로운 질문을 제공하세요. 
객관식 질문은 4개의 보기 문항을 포함하며, 새로운 질문을 연속적으로 이어가도록 하세요.
"""

# 대화 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 기록 저장 개수

def ask_deepseek(prompt):
    """DeepSeek-R1 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    messages = [{"role": "system", "content": system_instruction}]  # 시스템 메시지 추가
    
    for i in range(0, len(chat_history), 2):
        messages.append({"role": "user", "content": chat_history[i].replace("User: ", "")})
        messages.append({"role": "assistant", "content": chat_history[i + 1].replace("DeepSeek: ", "")})
    
    # 사용자의 새 질문 추가
    messages.append({"role": "user", "content": prompt})

    # 토큰 변환
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 종료 토큰 설정
    eos_token = tokenizer.eos_token_id

    # 모델 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=eos_token,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 응답 디코딩
    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # 새로운 대화 기록 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"DeepSeek: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_deepseek("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 2개 작성해줘."))

# 사용자가 선택했을 때의 응답 테스트
print(ask_deepseek("1번을 선택했어. 이에 대한 감정을 공감하면서, 다음 질문을 제시해줘."))  # 답변 반영

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")

c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.13s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
S

네, 기꺼이 도와드리겠습니다. 

1. 오늘 하루에 가장 기분 좋은 순간은 언제였나요?
   A. 아침 일어나서
   B. 아침 커피를 마시던 시기
   C. 업무 중에
   D. 저녁 식사 후

2. 오늘 하루에 가장 힘들었던 순간은 언제였나요?
   A. 출근하는 길에
   B. 중요한 회의 중에
   C. 점심시간에
   D. 퇴근 후

이제 당신이 선택한 답변에 대해 이야기해주실 수 있으실까요? 가장 기분 좋은 순간과 가장 힘들었던 순간에 대해 간단히 적어보시면 좋을 것 같아요. 

그 다음에 새로운 질문을 준비해드리겠습니다:
1. 기분 좋은 순간에서 느꼈던 감정은 무엇이었나요? 
   A. 행복
   B. 안정감
   C. 만족감
   D. 기쁨

2. 힘들었던 순간에서 겪었던 감정은 무엇이었나요?
   A. 스트레스
   B. 좌절감
   C. 불안
   D. 분노

당신의 답변을 기다리겠습니다.
아, 아침 일어나서 기분 좋았셨군요. 새벽의 평화와 고요함, 그리고 하루를 시작하는 활력이 느껴지실 거예요. 

그런 순간에는 일상의 소소한 기쁨에 감사하게 느끼시죠? 

다음 질문을 제시해 보겠습니다:

1. 오늘 아침에 가장 기분 좋게 느꼈던 이유는 무엇이었나요?
   A. 좋은 잠을 자서
   B. 좋아하는 음악을 들으며
   C. 창문을 통해 들어온 따뜻한 햇살
   D. 아침 식사를 맛있게 먹었기 때문

당신의 선택에 대해 적어주시면 그에 맞는 새로운 질문을 준비해 드리겠습니다.

=== 대화 내역 ===
User: 오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 2개 작성해줘. 
----------------------
DeepSeek: 네, 기꺼이 도와드리겠습니다. 

1. 오늘 하루에 가장 기분 좋은 순간은 언제였나요?
   A. 아침 일어나서
   B. 아침 커피를 마시던 시기
   C. 업무 중에
   D. 저녁 식사 후

2. 오늘 하루에 가장 힘들었던 순간은 언제였나요?
   A. 출근하는 길에
   B. 중